In [1]:
import pandas as pd

## str简介

1. pandas中的str对象方法，照搬了很多标准库中str方法，所以跟标准库中的str模块方法同名且功能一致。如：capitalize，title，upper，lower等等
2. str对象是定义在Index或Series上的属性
3. 可以使用索引器对字符串进行序列化的操作
4. 可以使用[正则匹配](https://github.com/cdoco/learn-regex-zhhttps://github.com/cdoco/learn-regex-zh)

## 常用函数

* **upper**：产生的新字符串，所有字符全转成大写
* **lower**：产生的新字符串，所有字符全转成小写
* **title**：产生的新字符串，每个单子都首字母大写
* **capitalize**：产生的新字符串，首字母大写
* **swapcase**：产生的新字符串，大小写转换

* **count**
* **len**

In [2]:
s = pd.Series(["a", None, "b"], dtype="object")
s.str.count('a')

0    1.0
1    NaN
2    0.0
dtype: float64

In [5]:
s.str.len()

0    1.0
1    NaN
2    1.0
3    NaN
dtype: float64

* **isdigit**

In [4]:
s = pd.Series(["a", None, "b", 1], dtype="object")
s.str.isdigit()

0    False
1     None
2    False
3      NaN
dtype: object

* **to_numeric**：是否为数值型，其主要参数包括 errors 和 downcast 分别代表了非数值的处理模式和转换类型。其中，对于不能转换为数值的有三种 errors 选项， raise, coerce, ignore 分别表示直接报错、设为缺失以及保持原来的字符串。

In [21]:
s = pd.Series(['1', '2.2', '2e', '??', '-2.1', '0'])

In [22]:
pd.to_numeric(s, errors='ignore')

0       1
1     2.2
2      2e
3      ??
4    -2.1
5       0
dtype: object

In [24]:
pd.to_numeric(s, errors='coerce')

0    1.0
1    2.2
2    NaN
3    NaN
4   -2.1
5    0.0
dtype: float64

In [26]:
#  快速查看非数值的行或数值行
s[pd.to_numeric(s, errors='coerce').isna()]

2    2e
3    ??
dtype: object

* **to_numpy**：转为numpy数组

In [10]:
s = pd.Series(["a", None, "b", 1], dtype="object")
s.to_numpy()

array(['a', None, 'b', 1], dtype=object)

## 文本处理

### 1. <font color=red size=5>**str.split**</font> 字符串拆分
    * 第一个参数为正则表达式
    * 第二个可选参数n为拆分次数
    * 第三个可选参数expand=True为是否展开为多个列

In [6]:
s = pd.Series(['test_1', 'test-2', 'test_1_1'])
s

0      test_1
1      test-2
2    test_1_1
dtype: object

In [7]:
s.str.split('_')

0       [test, 1]
1        [test-2]
2    [test, 1, 1]
dtype: object

In [9]:
s.str.split('_').get(2)

['test', '1', '1']

In [10]:
s.str.split('_|-', n=1)

0      [test, 1]
1      [test, 2]
2    [test, 1_1]
dtype: object

In [11]:
s.str.split('_|-', n=1, expand=True)

,0,1
0,test,1
1,test,2
2,test,1_1


### 2. <font color=red size=5>**str.join和str.cat**</font> 字符串合并

    * str.join表示用连接符把Series中的字符串列表连接起来
    * str.cat表示合并两个Series序列，参数有连接符sep,连接形式join以及缺失值替代符号na_rep

In [16]:
s = pd.Series([['A', 'a'], ['B', 'b'], ['c', 1]])
s

0    [A, a]
1    [B, b]
2    [c, 1]
dtype: object

In [18]:
#  如果列表中出现了非字符串元素则返回缺失值
s.str.join('_')

0    A_a
1    B_b
2    NaN
dtype: object

In [23]:
s1 = pd.Series(['A', 'B'])
s2 = pd.Series(['a', 'b'])
s1.str.cat(s2, sep='_')

0    A_a
1    B_b
dtype: object

In [24]:
s2.index = [1, 2]

In [26]:
s1.str.cat(s2, sep='_', join='outer', na_rep='NNNNN')

0    A_NNNNN
1        B_a
2    NNNNN_b
dtype: object

### 3. <font color=red size=5>**字符串匹配**</font> 

返回布尔值匹配，支持正则匹配
* **str.contains**
* **str.startswith**
* **str.endswith**
* **str.match**  

返回索引匹配，rfind是从右到左第一次匹配的位置索引，未找到时返回-1，不支持正则匹配
* **str.find, str.rfind**


In [29]:
s = pd.Series(['#my cat', 'he is fat', 'railway station'])

In [30]:
s.str.contains('\s\wat')

0     True
1     True
2    False
dtype: bool

In [31]:
s.str.startswith('#')

0     True
1    False
2    False
dtype: bool

In [32]:
s.str.endswith('t')

0     True
1     True
2    False
dtype: bool

In [33]:
s.str.match('#|h')

0     True
1     True
2    False
dtype: bool

In [34]:
s.str.find('my')

0    1
1   -1
2   -1
dtype: int64

In [36]:
s.str.rfind('st')

0   -1
1   -1
2    8
dtype: int64

### 4. <font color=red size=5>**字符串替换**</font> 

* **str.replace**

支持正则匹配

In [37]:
s = pd.Series(['a_1_b', 'c_?'])
s.str.replace('\d|\?', 'new', regex=True)

0    a_new_b
1      c_new
dtype: object

支持分组匹配—**group**函数

In [40]:
s = pd.Series(['上海市黄浦区方浜中路249号', '上海市宝山区密山路5号', '北京市昌平区北农路2号'])

In [41]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
city = {'上海市': 'shanghai', '北京市': 'beijing'}
district = {'昌平区': 'CP District', '黄浦区': 'HP District', '宝山区': 'BS District'}
road = {'方浜中路': 'Mid Fangbin Road', '密山路': 'Mishan Road', '北农路': 'Beinong Road'}

In [53]:
def my_func(m):
    str_city = city[m.group(1)]
    str_district = district[m.group(2)]
    str_road = road[m.group(3)]
    str_no = 'No.' + m.group(4)[:-1]
    return ' '.join([str_city, str_district, str_road, str_no])

In [54]:
s.str.replace(pat, my_func, regex=True)

0    shanghai HP District Mid Fangbin Road No.249
1           shanghai BS District Mishan Road No.5
2           beijing CP District Beinong Road No.2
dtype: object

In [55]:
s.str.replace(pat, )

0    上海市黄浦区方浜中路249号
1       上海市宝山区密山路5号
2       北京市昌平区北农路2号
dtype: object

In [56]:
#  ?P<value>的意思就是命名一个名字为value的组
>>> pat = r"(?P<one>\w+) (?P<two>\w+) (?P<three>\w+)"  # 匹配的是\w+ \w+ \w+
>>> repl = lambda m: m.group('two').swapcase()
>>> pd.Series(['One Two Three', 'Foo Bar Baz']).str.replace(pat, repl)

<ipython-input-56-e56352b2242a>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  pd.Series(['One Two Three', 'Foo Bar Baz']).str.replace(pat, repl)


0    tWO
1    bAR
dtype: object

In [58]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
def my_func2(m):
    str_city = city[m.group('市名')]
    str_district = district[m.group('区名')]
    str_road = road[m.group('路名')]
    str_no = 'No.' + m.group('编号')[:-1]
    return ' '.join([str_city, str_district, str_road, str_no])
s.str.replace(pat, my_func2, regex=True)

0    shanghai HP District Mid Fangbin Road No.249
1           shanghai BS District Mishan Road No.5
2           beijing CP District Beinong Road No.2
dtype: object

In [57]:
>>> import re
>>> regex_pat = re.compile(r'FUZ', flags=re.IGNORECASE)
>>> pd.Series(['foo', 'fuz', np.nan]).str.replace(regex_pat, 'bar')

0    foo
1    bar
2    NaN
dtype: object

### 5. <font color=red size=5>**字符串提取**</font> 

* **str.extract** ：返回具体元素，只匹配一次，没有搜索到的默认返回NA，参数expand默认为True，返回数据框，False时如果匹配到单个返回Series
* **str.extract**
* **str.extractall**：把所有符合条件的模式全部匹配出来，多级索引的形式保存
* **str.findall**：功能类似extractall，区别在于以列表的形式保存匹配结果

In [14]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(
    r"(?P<letter>[ab])(?P<digit>\d)", expand=False)

,letter,digit
0,a,1
1,b,2
2,<NA>,<NA>


In [20]:
pd.Series(["a1", "b2", "c3"], dtype="string").str.extract(
    r"(?P<letter>[ad])", expand=False)

0       a
1    <NA>
2    <NA>
Name: letter, dtype: string

In [59]:
pat = '(\w+市)(\w+区)(\w+路)(\d+号)'
s.str.extract(pat)

,0,1,2,3
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


通过子组的命名，可以直接对新生成 DataFrame 的列命名：

In [61]:
pat = '(?P<市名>\w+市)(?P<区名>\w+区)(?P<路名>\w+路)(?P<编号>\d+号)'
s.str.extract(pat)

,市名,区名,路名,编号
0,上海市,黄浦区,方浜中路,249号
1,上海市,宝山区,密山路,5号
2,北京市,昌平区,北农路,2号


In [63]:
s = pd.Series(['A135T15,A26S5','B674S2,B25T6'], index = ['my_A','my_B'])
pat = '[A|B](\d+)[T|S](\d+)'
s.str.extractall(pat)

0   1
     match         
my_A 0      135  15
     1       26   5
my_B 0      674   2
     1       25   6

In [65]:
s.str.findall(pat)

my_A    [(135, 15), (26, 5)]
my_B     [(674, 2), (25, 6)]
dtype: object